In [1]:
'''
数据过大，且文档格式为某一只股票在一段时间内的5分钟k线数据，所以分批读入；
2017年年末到2022年年末，5年时间一共调整了21次中证1000成分股，手动5次读取数据
'''

In [1]:
import pandas as pd
import numpy as np
import os
import pyarrow
import math

In [2]:
#设置数据储存位置
read_path=r'I:\PG\QInvestment\project\zz1000_5min'

In [3]:
#读入数据储存地址
def getPathTime(path):
    allpaths=[]
    timelist=[]
    files=os.listdir(path)
    for file_path in files:
        temptime=file_path[10:-3]
        new_path=os.path.join(path,file_path)
        timelist.append(temptime)
        allpaths.append(new_path)
    return allpaths,timelist

In [4]:
all_read_path,timelist=getPathTime(read_path)

In [5]:
#按时间读入数据，用数据框查看多少个时间段
df_timelist=pd.DataFrame(timelist)
timeNums=df_timelist.drop_duplicates().sort_values(by=0).reset_index(drop=True)
datelist=df_timelist.drop_duplicates().sort_values(by=0)[0].to_list()

In [6]:
prepare_list = locals()#获取当前位置前的全部局部变量
grouplist=[]
for ii in range(timeNums.shape[0]):
    groupname='group'+str(ii+1)
    grouplist.append(groupname)
    prepare_list[groupname]=[]
    for path in all_read_path:
        if datelist[ii] in path:
            prepare_list[groupname].append(path)

In [7]:
def getDataframe(paths):#paths是列表
    dfsum=pd.DataFrame(data=None)
    for path in paths:
        tempdata=pd.read_parquet(path,columns=['date','time','code','high','low'],engine='auto')
        dfsum=pd.concat([dfsum,tempdata])
    return dfsum

In [8]:
#以时间分21组，每组1000个函数，读取数据21次
#数据太大手动拆分,分5批读入
prepare_df_list = locals()#获取当前位置前的全部局部变量
data_name_list=[]

In [9]:
##读取前4组
for ii in range(4):
    dataname='data'+str(ii+1)
    data_name_list.append(dataname)
    prepare_df_list[dataname]=pd.DataFrame(getDataframe(prepare_list[grouplist[ii]]))#变量名是列表元素，不代表储存值的列表名

In [ ]:
##读取5-8组
#for ii in np.arange(4,8,1):
 #   dataname='data'+str(ii+1)
  #  data_name_list.append(dataname)
   # prepare_df_list[dataname]=pd.DataFrame(getDataframe(prepare_list[grouplist[ii]]))#变量名是列表元素，不代表储存值的列表名

In [ ]:
##读取9-12组
#for ii in np.arange(8,12,1):
 #   dataname='data'+str(ii+1)
 #   data_name_list.append(dataname)
 #   prepare_df_list[dataname]=pd.DataFrame(getDataframe(prepare_list[grouplist[ii]]))#变量名是列表元素，不代表储存值的列表名

In [ ]:
##读取13-18组
#for ii in np.arange(12,18,1):
 #   dataname='data'+str(ii+1)
 #   data_name_list.append(dataname)
 #   prepare_df_list[dataname]=pd.DataFrame(getDataframe(prepare_list[grouplist[ii]]))#变量名是列表元素，不代表储存值的列表名

In [ ]:
##读取19-21组

#for ii in np.arange(18,21,1):
 #   dataname='data'+str(ii+1)
  #  data_name_list.append(dataname)
   # prepare_df_list[dataname]=pd.DataFrame(getDataframe(prepare_list[grouplist[ii]]))#变量名是列表元素，不代表储存值的列表名

In [10]:
#处理时间
def timeProcess(data):
    tempdata=data.reset_index(drop=True)
    tempdata['time']=tempdata['time'].apply(lambda x:x[8:12])#变为0935 4位格式
    return tempdata

In [11]:
#去掉前15分钟以及最后15分钟的数据
def dropData15(data):   
    #前15分钟
    index1=data[(data['time']=='0935') | (data['time']=='0940') | (data['time']=='0945')].index.tolist()
    #后15分钟
    index2=data[(data['time']=='1445') | (data['time']=='1450') | (data['time']=='1455') | (data['time']=='1500')].index.tolist()
    index=index1+index2
    data1=data.drop(index)
    data1=data1.reset_index(drop=True)
    return data1

In [12]:
#得到日内波动率
def Parkinson(data):
    timelist=[]
    codelist=[]
    timeStampNum=np.array(data.groupby(['date','code']).count()['time'])#日内时间点为41
    tempcol=np.array(data.groupby(['date','code']).count()['time'].index)
    data.iloc[:,-3:]=data.iloc[:,-3:].astype(float)#将low/high/close转为float
    data['highLowDaily']=data['high']/data['low']
    data['highLowDaily']=data['highLowDaily'].apply(lambda x: math.log(x,math.e))
    data['highLowDaily']=data['highLowDaily'].apply(lambda x: x**2)
    temp_variance=np.array(data.groupby(['date','code']).sum()['highLowDaily'])
    parkinsonV=(temp_variance/(4*timeStampNum*math.log(2,math.e)))**0.5
    for ii in tempcol:
        timelist.append(ii[0])
        codelist.append(ii[1])
    data_v=pd.DataFrame({'date':timelist,
                         'code':codelist,
                         'variance':parkinsonV})
    return data_v

In [13]:
#2023格式不同
def Parkinson1(data):
    timelist=[]
    codelist=[]
    timeStampNum=np.array(data.groupby(['date','code']).count()['time'])#日内时间点为41
    tempcol=np.array(data.groupby(['date','code']).count()['time'].index)
    data.iloc[:,-2:]=data.iloc[:,-2:].astype(float)#将low/high/close转为float
    data['highLowDaily']=data['high']/data['low']
    data['highLowDaily']=data['highLowDaily'].apply(lambda x: math.log(x,math.e))
    data['highLowDaily']=data['highLowDaily'].apply(lambda x: x**2)
    temp_variance=np.array(data.groupby(['date','code']).sum()['highLowDaily'])
    parkinsonV=(temp_variance/(4*timeStampNum*math.log(2,math.e)))**0.5
    for ii in tempcol:
        timelist.append(ii[0])
        codelist.append(ii[1])
    data_v=pd.DataFrame({'date':timelist,
                         'code':codelist,
                         'variance':parkinsonV})
    return data_v

In [14]:
#对日内波动率进行标准化处理
def standardProcess(dataV):
    meanDaily=dataV.groupby(['date']).mean()['variance']
    stdDaily=dataV.groupby(['date']).std()['variance']
    tempDaily=pd.concat([meanDaily,stdDaily],axis=1,keys=['mean','varianceDaily'])
    standVariance=dataV.merge(tempDaily,on='date',how='left')
    dataV['variance']=(standVariance['variance']-standVariance['mean'])/standVariance['varianceDaily']
    return dataV

In [16]:
#组合每一批计算得到的日内波动率
datagroup=pd.DataFrame(data=None)
for dataname in data_name_list:
    tempdata=timeProcess(prepare_df_list[dataname])
    tempdata1=dropData15(tempdata)
    parkinsonData=Parkinson1(tempdata1)
    dataStandardV=standardProcess(parkinsonData)
    datagroup=pd.concat([datagroup,dataStandardV])

C:\Users\user\AppData\Local\Temp\ipykernel_10572\3306785750.py:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  temp_variance=np.array(data.groupby(['date','code']).sum()['highLowDaily'])
C:\Users\user\AppData\Local\Temp\ipykernel_10572\1052861444.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  meanDaily=dataV.groupby(['date']).mean()['variance']
C:\Users\user\AppData\Local\Temp\ipykernel_10572\1052861444.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns

In [17]:
#手动验证5分钟K线数据是否有缺失值
tempdata1.shape[0]/parkinsonData.shape[0]#是否为41

41.0

In [ ]:
#手动写入文件
datagroup.to_csv('I:\PG\QInvestment\project\datagroup1.csv')
#datagroup.to_csv('I:\PG\QInvestment\project\datagroup2.csv')
#datagroup.to_csv('I:\PG\QInvestment\project\datagroup3.csv')
#datagroup.to_csv('I:\PG\QInvestment\project\datagroup4.csv')
#datagroup.to_csv('I:\PG\QInvestment\project\datagroup5.csv')

In [ ]:
#读取2023年数据
read_path1=r'I:\PG\QInvestment\project\zz1000_5min_high_low_2023'
data2023=pd.read_parquet(read_path1,engine='auto')

In [ ]:
#单独计算2023年
tempdata=timeProcess(data2023)
tempdata1=dropData15(tempdata)
parkinsonData=Parkinson1(tempdata1)
datagroup=standardProcess(parkinsonData)

In [ ]:
datagroup.to_csv('I:\PG\QInvestment\project\datagroup6.csv')

In [ ]:
#单独计算2017年12月前40天数据
read_path2=r'I:\PG\QInvestment\project\zz1000_5min_high_low_2017.pq'
data2017=pd.read_parquet(read_path2,engine='auto')

In [ ]:
tempdata=timeProcess(data2017)
tempdata1=dropData15(tempdata)
parkinsonData=Parkinson(tempdata1)
datagroup=standardProcess(parkinsonData)

In [ ]:
datagroup.to_csv('I:\PG\QInvestment\project\datagroup0.csv')